In [2]:
import scantelligent.functions as st
from time import sleep
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from datetime import datetime

Start a session

In [3]:
session_path, images_path, logfile_path, parameters = st.initialize()

22:40:30  Scantelligent session initialized. Scantelligent log file created on 2025-10-21 at 22:40:30
          Directory 'C:\Data\Peter\WS2\101525\Extracted Files' created successfully or already existed.
{'z_averaging_time': 0.10000000149011612, 'z_offset': -1.4999999853326784e-10, 'initial_settling_time': 1.0, 'maximum_slew_rate': 4.000000953674316, 'settling_time': 0.019999999552965164, 'integration_time': 0.03999999910593033, 'end_settling_time': 9.999999747378752e-05, 'z_control_time': 0.0}
22:40:30  Report of parameters:

          Tip parameters:
            Location (x_tip, y_tip, z_tip) = (-350.0, -186.237, -198.0) nm. Tip displacement speed (v_tip) = 202 nm/s.
          Bias parameters:
            Bias voltage (V) = 1.8 V.
            Lockin modulation voltage (V_ac) = 0.05 V; Lockin modulation frequency (f_ac) = 683.0 Hz; Lockin phase (phase_ac) = -33.223 degree.
            Lockin is switched off (lockin_status = 0).
          Z controller parameters:
            Feedback

Manual log entries

In [3]:
message = "This is a manual entry. You can use this to add timestamped comments to the active logfile."
st.logprint(message, logfile = logfile_path)

10:06:26  This is a manual entry. You can use this to add timestamped comments to the active logfile.


Simple auto approach

In [14]:
st.auto_approach(I_approach = 100, p_gain_approach = 30, t_const_approach = 167, land_tip = True, timeout = 1000, z_voltage = 220, adjust_percentile = 60)
st.scan_control(action = "start", scan_direction = "down", monitor = False)

18:08:59  Tip withdrawn.
18:08:59  Current setpoint changed from I = 20.0 pA to I = 100 pA.
18:08:59  Gains changed from p_gain = 10.0 pm to p_gain = 30.0 pm; t_const = 167 us to t_const = 167 us.
18:08:59  Auto approach initiated.
18:09:00  Tip tracker started.
18:12:06  Tip deemed to be at rest. Exiting tip tracker.
18:12:06  Surface detected.
18:12:07  Tip withdrawn.
18:12:07  Current setpoint changed from I = 100.0 pA to I = 20.0 pA.
18:12:07  Gains changed from p_gain = 30.0 pm to p_gain = 10.0 pm; t_const = 167 us to t_const = 167 us.
18:12:08  Lockin autophased at a frequency of 683.0 Hz. Demodulator phase set to phi = -33.228 degree.
18:12:08  z controller switched on.
18:12:17  Tip landed too low in the z range (-163.102 nm < -39.6 nm). Adjusting by going down one coarse step.
18:12:17  Tip withdrawn.
18:12:17  z controller switched on.
18:12:25  Tip landed too low in the z range (-84.694 nm < -39.6 nm). Adjusting by going down one coarse step.
18:12:26  Tip withdrawn.
18:12:2

Simple coarse move in any direction

In [ ]:
for iteration in range(10):
    print(f"Iteration number {iteration}")
    st.move_over(direction = "east", steps = 1250, z_steps = 18, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
    st.scan_control(action = "start", scan_direction = "down", monitor = True)
    print(f"Iteration number {iteration + .5}")
    st.move_over(direction = "north", steps = 1250, z_steps = 18, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
    st.scan_control(action = "start", scan_direction = "down", monitor = True)

Iteration number 0
22:40:34  Scan stopped.
22:40:34  Tip withdrawn.
22:40:34  Coarse motion: 18 steps in the Z+ direction.
22:40:36  Coarse motion: 1250 steps in the X+ direction.
22:40:36  Tip withdrawn.
22:40:36  Current setpoint changed from I = 20.0 pA to I = 100 pA.
22:40:36  Gains changed from p_gain = 10.0 pm to p_gain = 30.0 pm; t_const = 167 us to t_const = 167 us.
22:40:36  Auto approach initiated.
22:40:36  Tip tracker started.
22:41:03  Tip deemed to be at rest. Exiting tip tracker.
22:41:03  Surface detected.
22:41:03  Tip withdrawn.
22:41:03  Current setpoint changed from I = 100.0 pA to I = 20.0 pA.
22:41:03  Gains changed from p_gain = 30.0 pm to p_gain = 10.0 pm; t_const = 167 us to t_const = 167 us.
22:41:05  Lockin autophased at a frequency of 683.0 Hz. Demodulator phase set to phi = -33.234 degree.
22:41:05  z controller switched on.
22:41:13  Tip landed too low in the z range (-158.031 nm < -99.0 nm). Adjusting by going down one coarse step.
22:41:13  Tip withdrawn

In [ ]:
st.scan_control(action = "start", scan_direction = "up", monitor = True)
st.move_over(direction = "north", steps = 150, z_steps = 15, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "down", monitor = True)
st.move_over(direction = "east", steps = 150, z_steps = 15, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "up", monitor = True)
st.move_over(direction = "north", steps = 150, z_steps = 15, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "down", monitor = True)
st.move_over(direction = "west", steps = 150, z_steps = 15, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "up", monitor = True)
st.move_over(direction = "north", steps = 1500, z_steps = 35, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "down", monitor = True)
st.move_over(direction = "east", steps = 1500, z_steps = 35, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 240, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "up", monitor = True)

In [ ]:
st.auto_approach(I_approach = 100, p_gain_approach = 30, t_const_approach = 167, land_tip = True, timeout = 1000, z_voltage = 220, adjust_percentile = 60)
st.scan_control(action = "start", scan_direction = "down")

In [13]:
st.coarse_move(direction = "approach", steps = 150)

18:08:39  Coarse motion: 150 steps in the Z- direction.


Some scanning, coarse moving, changing parameters

In [ ]:
st.scan_control(action = "start", scan_direction = "up")
st.move_over(direction = "north", steps = 20, z_steps = 10, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 220, velocity_threshold = .02, adjust_percentile = 60)
st.change_bias(-2)
st.change_scan_window(center = [30, -100], size = 120, grid = 260, angle = 22)
st.scan_control(action = "start", scan_direction = "down")

In [ ]:
st.scan_control(action = "start", scan_direction = "down", velocity_threshold = .2)

In [ ]:
st.scan_control(action = "stop")

Change functions. They return the old parameters as tuples. Any parameters that are left None or are omitted will remain unchanged.

In [ ]:
V_old = st.change_bias(V = None)
I_old, p_gain_old, t_const_old = st.change_feedback(I = None, p_gain = None, t_const = None)
center_old, size_old, grid_old, angle_old = st.change_scan_window(size = None, grid = None, angle = None)

Tip tracker. Make it time out at 20 seconds. Then return and visualize the x(t), y(t) and z(t) trajectories, with one data point generated every 0.5 seconds. Meanwhile, calculate the surface roughness every 2 seconds.

In [ ]:
txyz = st.tip_tracker(sampling_time = .5, exit_when_still = False, timeout = 20, tracking_info = False, monitor_roughness = True, measurement_interval = 2)
plt.plot(txyz[1], txyz[2])
#plt.plot(txyz[0], txyz[2])
#plt.plot(txyz[0], txyz[3])
plt.show()

Image analysis. Load all files in the current working directory of Nanonis. Change the path argument in get_file_data to get sxm files from another folder

In [3]:
session_path = st.get_session_path()
file_data = st.get_file_data(session_path = session_path)
sxm_files = file_data.image_files
sxm_files_in_folder = [file.split("\\")[-1] for file in sxm_files] # This removes the directory from the path

print("The session path is: " + session_path)
print("SXM files found in this directory: " + "; ".join(sxm_files_in_folder))

The session path is: C:\Data\Qile\082025
SXM files found in this directory: img_0001.sxm; img_0002.sxm; img_0003.sxm; img_0004.sxm; img_0005.sxm; img_0006.sxm; img_0007.sxm; img_0008.sxm; img_0009.sxm; img_0010.sxm; img_0011.sxm; img_0012.sxm; img_0013.sxm; img_0014.sxm; img_0015.sxm; img_0016.sxm; img_0017.sxm; img_0018.sxm; img_0019.sxm; img_0020.sxm; img_0021.sxm; img_0022.sxm; img_0023.sxm; img_0024.sxm; img_0025.sxm; img_0026.sxm; img_0027.sxm; img_0028.sxm; img_0029.sxm; img_0030.sxm; img_0031.sxm; img_0032.sxm; img_0033.sxm; img_0034.sxm; img_0035.sxm; img_0036.sxm; img_0037.sxm; img_0038.sxm; img_0039.sxm; img_0040.sxm; img_0041.sxm; img_0042.sxm; img_0043.sxm; img_0044.sxm; img_0045.sxm; img_0046.sxm; img_0047.sxm; img_0048.sxm; img_0049.sxm; img_0050.sxm; img_0051.sxm; img_0052.sxm; img_0053.sxm; img_0054.sxm; img_0055.sxm; img_0056.sxm; img_0057.sxm; img_0058.sxm; img_0059.sxm; img_0060.sxm; img_0061.sxm; img_0062.sxm; img_0063.sxm; img_0064.sxm; img_0065.sxm; img_0066.sxm; 

Get the data for a scan. First make sure the scan file exists and contains the target channel. Then, check how clipping and background subtraction affects the scan image.

In [ ]:
file_no = 10
target_channel = "Z"
dirxn = 0 # dirxn = 0 means forward; dirxn = 1 means backward
clip_percentiles = [20, 80]

if file_no > len(sxm_files) - 1: print("Error: This file does not exist.")
else:
    scan_data = st.get_scan(sxm_files[file_no]) # Import all the scan data using Nanonispy
    V = scan_data.V
    size = scan_data.header.get("scan_range", [100, 100])
    
    channels = scan_data.channels
    find_channel = np.where(np.array(channels) == target_channel)[0]
    
    if len(find_channel) != 1: print("Error: The target channel was not found")
    else:
        channel_index = find_channel[0]
        filename = images_path + "\\" + channels[channel_index] + f"_{file_data.image_indices[file_no]:03d}.png"
        img_raw = scan_data.scans[channel_index, dirxn]
        
        clip_values = st.clip_range(img_raw, method = "percentiles", values = clip_percentiles)
        histogram = st.get_image_statistics(img_raw).histogram
        img_b = st.background_subtract(img_raw)
        clip_values_b = st.clip_range(img_b, method = "percentiles", values = clip_percentiles)
        histogram_b = st.get_image_statistics(img_b).histogram

        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(img_raw, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")
        ax[1].imshow(img_b, vmin = clip_values_b[0], vmax = clip_values_b[1], cmap = "gray")
        fig.set_size_inches(10, 5)
        plt.show()

        fig, ax = plt.subplots()
        ax.plot(-histogram[1], histogram[0] - np.mean(histogram[0]))
        ax.plot(histogram_b[1], histogram_b[0])
        print("Image histograms:")
        plt.show()
        
    print(filename)

Check the background subtraction and clipping

In [ ]:
background = True

clip_values = st.clip_range(img_raw, method = "percentiles", values = clip_percentiles)
histogram = st.get_image_statistics(img_raw).histogram
img_b = st.background_subtract(img_raw)
clip_values_b = st.clip_range(img_b, method = "percentiles", values = clip_percentiles)
histogram_b = st.get_image_statistics(img_b).histogram

if background:
    img = img_b
    clip_values = clip_values_b
else: img = img_raw

fig, ax = plt.subplots()
ax.imshow(img, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")
fig.set_size_inches(10, 5)
plt.show()
plt.imsave(filename, img, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")

print("Successfully saved " + filename)

In [9]:
from nanonisTCP import nanonisTCP
from nanonisTCP.ZController import ZController
from nanonisTCP.Motor import Motor
from nanonisTCP.AutoApproach import AutoApproach
from nanonisTCP.Scan import Scan
from nanonisTCP.Current import Current
from nanonisTCP.LockIn import LockIn
from nanonisTCP.Bias import Bias
from nanonisTCP.Signals import Signals
from nanonisTCP.Util import Util
from nanonisTCP.FolMe import FolMe
from nanonisTCP.BiasSpectr import BiasSpectr
from nanonisTCP.TipShaper import TipShaper
import nanonispy as nap
import numpy as np
from datetime import datetime
import time
from time import sleep
from pathlib import Path
from scipy.signal import convolve2d
import os
import socket

# Establish a TCP/IP connection
TCP_IP = "192.168.236.1"                           # Local host
TCP_IP = "127.0.0.1"
TCP_PORT = 6501                                    # Check available ports in NANONIS > File > Settings Options > TCP Programming Interface
version_number = 13520

In [11]:
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    sock.connect((TCP_IP, TCP_PORT))
    sock.close()
    nanonis_online = True
except socket.timeout:
    nanonis_online = False

print(f"Nanonis is {nanonis_online} online")

Nanonis is True online


In [22]:
from time import sleep
import os

macro_folder = "C:\\Scripts\\Scantelligent\\macros"
macro_file = "test1.txt"

def run_macro(macro_file):
    if not os.path.exists(macro_file):
        print("Error! Macro file does not exist!")
        return
    else:
        macro_file_basename = os.path.basename(macro_file)

        macro = []
        with open(macro_file, "r") as file:
            for line in file:
                macro.append(line.strip())

        # Test if the code compiles
        code_list = []
        for line in range(len(macro)):
            try:
                code_line = compile(macro[line], "<string>", "exec")
                code_list.append(code_line)
            except SyntaxError as e:
                print(f"Syntax error encountered during compilation: {e}")
                return
            except Exception as e:
                print(f"Error encountered during compilation: {e}")
                return
        
        # Execution
        error_count = 0
        for code_line in code_list:
            try:
                exec(code_line)
            except SyntaxError as e:
                print(f"Syntax error encountered during execution: {e}")
                error_count += 1
            except Exception as e:
                print(f"Error encountered during execution: {e}")
                error_count += 1
        
        print(f"Execution of macro {macro_file_basename} completed with {error_count} errors")

In [25]:
run_macro(macro_folder + "\\" + macro_file)

Hello world
This is a macro
Error encountered during execution: name 'slep' is not defined
Continue execution despite typo
This is a macro
Execution of macro test1.txt completed with 1 errors
